In [5]:
import numpy as np
import revreaddy as rdy
import logging
import datetime
import h5py as h5
import matplotlib.cm as cm

import matplotlib.pyplot as plt
reload(logging)
logging.basicConfig(
     format='%(asctime)s %(levelname)s: %(message)s',
     datefmt='%Y/%m/%d %I:%M:%S',
     level=logging.DEBUG
)

In [6]:
def simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.timestep = 0.02 # ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0., reactiondistance)
    sim.delete_all_particles()
    for _ in range(300):
         pos = np.random.random(3) * 9.9 - 4.95
         sim.add_particle(pos, 0)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 1)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 2)
    for _ in range(1):   
        pos = np.random.random(3) * 9.9 - 4.95
        sim.add_particle(pos, 2)
    sim.show_world()
    sim.delete_all_observables()
    d = datetime.datetime.now()
    time=str(getattr(d, 'year','month'))+str(getattr(d,'month'))+str(getattr(d,'day'))+str(getattr(d,'hour'))+str(getattr(d,'minute'))+str(getattr(d,'second'))
    numbers_a_name = time+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    numbers_b_name = time+"numbers_b_test1.h5"
    mean_e_name = time+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(1000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb ,fc



In [7]:
#hier kommt die berechnung des verhältnisses der reactionskoeffizienten und der Concentration vom enzym
def analyt(t,boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction):
    #a=((4*np.pi*reactiondistance**3)*(Diffusion_E+Diffusion_S))/((3*boxsize**3)*(Diffusion_E+Diffusion_P))
    k1=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreaction)+np.tanh(reactiondistance*np.sqrt(intrinsikreaction/(Diffusion_E+Diffusion_S))))
    #t=np.linspace(1,40)
    #vor und rück reaction möglich                                        
    #c=np.exp(-t*a)*fa['particleNumbers'][0]
    #nur vor reaction
    k11=intrinsikreaction*4*np.pi*reactiondistance**3/3
    c=fa['particleNumbers'][0]*np.exp((-t*k1*1)/(boxsize**3))
    c11=fa['particleNumbers'][0]*np.exp((-t*k11*1)/(boxsize**3))

    return t, c,c11

In [8]:
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [2,8,20,48]:
    boxsize=13.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=4
    intrinsikreaction=20
    colornow=next(colors)
    #falist=[]
    #for ii in range(1):
    fa,fb,fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction)
    #    falist.append(fa['particleNumbers'])

    #falist=np.array(falist)
    t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
    #paertnum=falist.mean(axis=0)   
    plt.plot(fa['times'], fa['particleNumbers'],color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    plt.plot(t,c,color=colornow)
    
plt.plot(t,c11,color=colornow)
plt.legend()

2016/05/19 10:23:16 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/19 10:23:28 INFO: Finished after 12.28099 seconds.
2016/05/19 10:23:28 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/19 10:23:40 INFO: Finished after 12.238943 seconds.
2016/05/19 10:23:40 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/19 10:23:53 INFO: Finished after 12.197386 seconds.
2016/05/19 10:23:53 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/19 10:24:05 INFO: Finished after 12.252394 seconds.


In [9]:
plt.plot(t,c11)
plt.plot(t,c)

In [10]:
t=np.array(fa['times'])
plt.plot(fa['times'],fc['meanSquaredDisplacements'])
plt.plot(fa['times'],t*2*3*20)

In [11]:
plt.show()

In [12]:
print range(1)

[0]
